In [1]:
import torch
from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
from models.ConditionalUNet1D import ConditionalUnet1D
from utils.tools import SO3_to_ortho6d, ortho6d_to_SO3, create_se3_matrix
import os, pickle
import numpy as np
from utils.core import compute_jacobian, compute_PMP

In [2]:
model = "DDPM"
model_path = "./params/pouring_dataset/" + model + "/model_ep5000.pt"
dataset_path = "./datasets/pouring_data/"
data_name = "3_water_200.pkl"
device = 'cuda'
num_diffusion_iters = 100
input_dim=9

In [3]:
# Load data
with open(os.path.join(dataset_path, data_name), 'rb') as f:
    data = pickle.load(f)

# Trajectory of bottle
traj = data['traj']
traj = traj@np.array([[
    [1., 0., 0., data['offset'][0]], 
    [0., 1., 0., data['offset'][1]], 
    [0., 0., 1., data['offset'][2]], 
    [0., 0., 0., 1.]]])

traj = torch.Tensor(traj).type(torch.float32).to(device)

rotation_matrices = traj[:, :3, :3]
translations = traj[:, :3, 3]

ortho6d = SO3_to_ortho6d(rotation_matrices)
traj = torch.cat([ortho6d, translations], dim=1)
traj = torch.unsqueeze(traj, 0)

print("traj.shape:",traj.shape)

traj.shape: torch.Size([1, 480, 9])


In [4]:
# Load model
noise_pred_net = ConditionalUnet1D(
    input_dim=input_dim,
    global_cond_dim=0
)

noise_pred_net = torch.load(model_path, map_location='cuda', weights_only=False)

noise_scheduler = DDPMScheduler(
    num_train_timesteps=num_diffusion_iters,
    # Squared cosine
    beta_schedule='squaredcos_cap_v2',
    # Clip output to [-1,1]
    clip_sample=True,
    prediction_type='epsilon'
)

_ = noise_pred_net.to(device)

In [5]:
jacobian_rank = []
for t in [x * 0.05 for x in range(1)]:
    timestep = int(t * num_diffusion_iters)

    # Calculate x_t
    noise = torch.randn(traj.shape, device=device)
    x_t = noise_scheduler.add_noise(
        traj, noise, torch.tensor([timestep], dtype=torch.long).to(device)
    )

    # Calculate PMP using the computed x_t
    x_t = x_t.detach().clone().requires_grad_(True)
    PMP = compute_PMP(x_t, timestep, noise_pred_net, noise_scheduler)

    # Compute Jacobian of PMP with respect to x_t
    jacobian = compute_jacobian(PMP, x_t)
    
    if t==0:
        print("PMP shape:", PMP.shape)
        print("Jacobian shape:", jacobian.shape)
    
    jacobian_rank.append(torch.linalg.matrix_rank(jacobian))

PMP shape: torch.Size([1, 480, 9])
Jacobian shape: torch.Size([4320, 4320])


In [11]:
print(jacobian)
print(jacobian_rank)
print(torch.linalg.matrix_rank(jacobian, tol=(0.10 * torch.norm(jacobian))))

tensor([[ 5.2832e-01,  5.5921e-04, -1.6786e-02,  ...,  1.9585e-04,
         -2.1029e-04, -2.9839e-04],
        [ 7.0624e-03,  4.7831e-01,  1.6009e-02,  ..., -2.4452e-04,
          1.7860e-04,  2.7131e-05],
        [-1.2148e-02,  2.0635e-02,  4.8762e-01,  ..., -4.3336e-05,
          8.5794e-05, -2.8382e-04],
        ...,
        [ 9.0713e-05,  1.6199e-06,  3.3362e-05,  ...,  5.6528e-01,
          1.8556e-02, -2.6615e-02],
        [-6.8748e-05,  5.6451e-05,  1.5919e-05,  ...,  2.3607e-02,
          5.7656e-01, -1.6356e-02],
        [-1.1103e-04,  1.4634e-04, -9.5587e-05,  ..., -1.1138e-02,
         -8.9605e-03,  5.0486e-01]], device='cuda:0')
[tensor(4294, device='cuda:0')]
tensor(15, device='cuda:0')
